In [1]:
# - *- coding: utf- 8*-
from LambdaNodeEmbedding import KerasEval
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.callbacks import History

#These are the edges between the nodes.
edgelist = np.array([
                    [0,1,1,1],
                    [0,0,0,1],
                    [0,1,0,0],
                    [0,0,0,0]
                    ], dtype='float32')

edgelist_ten = tf.keras.backend.variable (value=edgelist, dtype='float32')
#edge_len = tf.keras.backend.constant (value=edgelist_ten.shape[1], dtype='int32')
#at_least_one = tf.keras.backend.variable (value=1, dtype='int32')

def GetXDepth(matrix):
    if len(matrix.shape) > 0:
        return matrix.shape[0]
    else:
        return 1

def GetYDepth(matrix):
    if len(matrix.shape) > 1:
        return matrix.shape[1]
    else:
        return 1

def BuildZeroedConcatRow(edge_row, index):
    zero_calc = tf.zeros((index,), tf.float32)
    keep_calc =  tf.ones(((edge_row.shape[0]-index),), tf.float32)
    concat_calc = tf.concat([zero_calc,keep_calc],0)
    result = tf.multiply(concat_calc, edge_row) 
    return result

def IndexOutOfBounds(edge_row, index):
    return tf.cond(tf.less_equal(index, edge_row.shape[0]), 
                   lambda: BuildZeroedConcatRow(edge_row, index),
                   lambda: tf.zeros(edge_row.shape, tf.float32))

def ZerofyUnderIndexInRow(edge_row, index):
    return tf.cond(tf.less(index, at_least_one), 
                   lambda: edge_row,
                   lambda: IndexOutOfBounds(edge_row, index))

def ZerofyUnderPrincipalDiagonal(edge_emb, result_emb, iterator):
    
    #Condition => if current iteration is lower as max iterations
    def cond(edge_emb, result_emb, i):
         return tf.less(i, iterator)

    #Loop Body Processing Unit => get all connection dependencies
    def body(edge_emb, result_emb, i):
        result_row = ZerofyUnderIndexInRow(edge_emb[i,:], i)
        
        #Store result in tensor array result_emb
        result_emb = result_row.write(i, [result_row])
        
        #Add +1 to Iterator and return process result!
        return [edge_emb, result_emb, tf.add(i, 1)]

    #return the loop setup
    return tf.while_loop(cond, body, [edge_emb, result_emb, 0])


init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    my_tuple = [edgelist_ten,edgelist_ten]
    
    tf_add_test = tf.add_n(my_tuple)
    print('Result_T:', KerasEval(tf_add_test))
    
    print('')
    
    keras_sum_test = my_tuple[0] + my_tuple[1]
    print('Result_K: ', KerasEval(keras_sum_test))
    
    
    #print(KerasEval(ZerofyUnderPrincipalDiagonal(edgelist_ten,edgelist_ten, edge_len)))


Result_T: [[0. 2. 2. 2.]
 [0. 0. 0. 2.]
 [0. 2. 0. 0.]
 [0. 0. 0. 0.]]

Result_K:  [[0. 2. 2. 2.]
 [0. 0. 0. 2.]
 [0. 2. 0. 0.]
 [0. 0. 0. 0.]]


Using TensorFlow backend.
